In [1]:
import psi4
import numpy as np
np.set_printoptions(3, linewidth=100, suppress=True)    # when we inspect the vectors/matrices, use a prettier format for printing

In [2]:
def deltaV(mol2,dl):
    bs=psi4.core.BasisSet.build(mol2)
    mints2= psi4.core.MintsHelper(bs)
    V0=np.asarray(mints2.ao_potential())
    for i in range(mol2.natom()):
        mol2.set_nuclear_charge(i,mol2.charge(i)+dl[i])
    V1f=np.asarray(mints2.ao_potential())
    for i in range(mol2.natom()):
        mol2.set_nuclear_charge(i,mol2.charge(i))
    return (V1f-V0)

The geometry of the water molecule is set to a previously optimized one for the aug-cc-pVDZ basis. We use the keyword `symmetry c1` to tell Psi4 not to use the actual point group $C_{2v}$ in the computation: If Psi4 used symmetry, we would not be able to create correct matrices for use in `numpy`.

In [7]:
mol = psi4.geometry('''
    C  0  0  0
    O  0  0  1.1
    symmetry c1
''')
psi4.set_options({'basis': 'sto-3G'})

We are now ready to run the SCF cycle to converge the wavefunction. By telling Psi4 to `return_wfn=True`, we get back the converged wavefunction from which we can extract the pertinent matrices. We print out the energy such that we can compare it against a pre-computed reference. It needs not exactly match the reference, slight variations are OK.

In [8]:
scf_energy, wfn = psi4.energy('SCF', return_wfn=True)
print('SCF Energy: {}'.format(scf_energy))  

SCF Energy: -111.21922150738187


In [9]:
C = np.asarray(wfn.Ca())  #mo coefficients

In [10]:
F = np.asarray(wfn.Fa())    # Fa=Fb for RHF
F = np.dot(C.T, F).dot(C)  # To MO

In [11]:
mints = psi4.core.MintsHelper(wfn.basisset())  #integral helper

In [12]:
I = np.asarray(mints.ao_eri())   # 2e repulsion integrals 

In [13]:
# transform to mo(so in psi4 terms)
g_pqrs = np.einsum('pqrs,pt->tqrs', I, C, optimize=True)
g_pqrs = np.einsum('pqrs,qt->ptrs', g_pqrs, C, optimize=True)
g_pqrs = np.einsum('pqrs,rt->pqts', g_pqrs, C, optimize=True)
g_pqrs = np.einsum('pqrs,st->pqrt', g_pqrs, C, optimize=True)

We are now ready to build the orbital rotation matrices $\mathbf{A}$ and $\mathbf{B}$. Recall from above that

\begin{align}
    A_{iajb} &= \delta_{ij} F_{ab} - \delta_{ab} F_{ij} + 2 g_{aijb} - g_{jiab}
\end{align}

The Kronecker deltas are represented by a unit matrix `numpy.eye(...)` which has the same shape as the molecular Fock matrix.

In [14]:
A = np.einsum('ij,ab->iajb', np.eye(F.shape[0]), F, optimize=True)
A -= np.einsum('ab,ij->iajb', np.eye(F.shape[0]), F, optimize=True)
A += 2 * np.einsum('aijb->iajb', g_pqrs, optimize=True)
A -= np.einsum('jiab->iajb', g_pqrs, optimize=True)

The $\mathbf{B}$ matrix is computed from

\begin{align}
    B_{iajb} &= 2 g_{aijb} - g_{ajbi}
\end{align}

and thus only contains contributions from the electron repulsion integrals.

In [15]:
B = 2 * np.einsum('aijb->iajb', g_pqrs, optimize=True)
B -= np.einsum('ajbi->iajb', g_pqrs, optimize=True)

The electronic Hessian is then simply the sum of these two matrices. When searching the literature, keep in mind that some authors defined their $\mathbf{B}$ with the opposite sign. For them, the Hessian will be the difference of both orbital rotation matrices.

In [16]:
H = A + B

Actually, we do not need the full Hessian, only the parts where the inactive and active orbitals contribute are of importance (see also Helgaker, sections 10.1.2 and 10.2.2). Therefore, we will now aim to extract the relevant parts of the Hessian. We already hinted at this by using the indices $i,j$ for inactive and $a,b$ for virtual orbitals in the definition of the Hessian.

For this, we need to know the number of number of basis functions = number of MOs, the number of doubly occupied orbitals (for RHF, identical to the number of electrons with $\alpha$ spin as all are paired) and the number of virtual orbitals. The number of rotable orbitals, i.e. the possible combinations of inactive and active orbitals, can then be computed by simple multiplication.

In [17]:
nbf = wfn.nmo()
ndocc = wfn.nalpha()
nvirt = nbf - ndocc
nrot = ndocc * nvirt

The elements in the Hessian are sorted, the inactive orbitals come first and the virtual ones second. Therefore, the number of doubly occupied orbitals is the boundary between both, so that we can use `ndocc` for slicing. Remember that we need to slice such that we get

\begin{align}
    H_{iajb}
\end{align}

Finally, to bring the Hessian into matrix form for `numpy.linalg.solve`, we reshape it to be of 'number of rotable' $\times$ 'number of rotable' orbitals.

In [18]:
H = H[:ndocc, ndocc:, :ndocc, ndocc:]
H = H.reshape(nrot, nrot)

We now have everything in place except for the perturbations themselves! These dipole tensors are computed by the `MintsHelper` instance and subsequently scaled by $-2$ to account for the doubly occupied orbitals.

# ADD HERE THE ALCHEMICAL PERTURBATION !!!!!!!!!!!!!!!!!!!!!!!!!

In [19]:
V=np.asarray(mints.ao_potential())
mol.set_nuclear_charge(0,7.0)  #C->N
mol.set_nuclear_charge(1,7.0)  # O->N
V1=np.asarray(mints.ao_potential())
mol.set_nuclear_charge(0,6.0)
mol.set_nuclear_charge(1,8.0)
dV=V1-V

In [20]:
P = np.asarray(wfn.Da())  # 1e DM

In [21]:
np.einsum('ij,ij',P,2*dV) # *2 is for RHF, is the EPN  APDFT1 deriv

6.340160873184406

In [16]:
#tmp_tensors = mints.so_dipole()     # get the tensors in the spin-orbital basis
dipole_tensors = []
for tensor in tmp_tensors:
    tensor.scale(-2)    # use the correct prefactor
    dipole_tensors.append(np.asarray(tensor))   # convert psi4's internal tensors to a numpy vector
for i in range(len(dipole_tensors)):
    dipole_tensors[i] = np.dot(C.T, dipole_tensors[i]).dot(C)    #to mo
    dipole_tensors[i] = dipole_tensors[i][:ndocc, ndocc:].ravel()  #ravelled for the solver

In [52]:
# Overwrite with the alchemical pertubation
dV_mo=np.dot(C.T, dV).dot(C)* 2  # important -2 for RHF
dipole_tensors=[dV_mo[:ndocc, ndocc:].ravel() ]   # overlap dipole tensor with apdft perturbation

We are finally ready to solve the linear response equations! For each perturbation, solve

\begin{align}
    H_{ia,jb} x_{jb} &= -f_{ia}
\end{align}

and store the resultant responses in `responses`.

In [53]:
responses = []
for perturbation in dipole_tensors:
    responses.append(np.linalg.solve(H, -perturbation))

The static polarizability tensor can be computed by dotting the perturbation vectors with the reponse vectors.

\begin{align}
    \alpha_{ij} &= - \mathbf{f}_i \mathbf{x}_j
\end{align}

This is exactly how the polarizability tensor is usually defined: It tells us how the system responds in $i$-direction to a perturbation along $j$.

In [54]:
#from pyscf
U=np.array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  3.93433594e-17, -1.57981413e-17,
         2.48052311e-02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  7.48935576e-17, -6.63967033e-18,
         2.97452931e-02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -3.94562190e-16,  5.46338024e-17,
        -1.42529802e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -1.13585752e-15,  9.76951215e-17,
         6.97260553e-02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -3.47947427e-01, -4.16764621e-03,
        -3.16709712e-16],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  4.16764621e-03, -3.47947427e-01,
        -4.61062817e-17],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -3.82058304e-15,  1.06302473e-15,
         6.11298241e-02],
       [-3.93433594e-17, -7.48935576e-17,  3.94562190e-16,
         1.13585752e-15,  3.47947427e-01, -4.16764621e-03,
         3.82058304e-15,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 1.57981413e-17,  6.63967033e-18, -5.46338024e-17,
        -9.76951215e-17,  4.16764621e-03,  3.47947427e-01,
        -1.06302473e-15,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [-2.48052311e-02, -2.97452931e-02,  1.42529802e-01,
        -6.97260553e-02,  3.16709712e-16,  4.61062817e-17,
        -6.11298241e-02,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00]])

In [55]:
responses[0],len(responses[0]) #len = 21 (7occ*3vir)

(array([ 0.   , -0.   , -0.05 , -0.   , -0.   ,  0.059, -0.   ,  0.   ,  0.285, -0.   ,  0.   ,
         0.139,  0.   ,  0.696, -0.   ,  0.696, -0.   ,  0.   , -0.   , -0.   , -0.122]),
 21)

In [56]:
U[:ndocc, ndocc:].ravel()

array([ 0.   , -0.   ,  0.025,  0.   , -0.   ,  0.03 , -0.   ,  0.   , -0.143, -0.   ,  0.   ,
        0.07 , -0.348, -0.004, -0.   ,  0.004, -0.348, -0.   , -0.   ,  0.   ,  0.061])

In [57]:
np.dot(dV_mo[:ndocc, ndocc:].ravel(), responses[0])

-1.9958869778638308

In [59]:
polarizabilities = np.zeros((3, 3))
for i in range(1):
    for j in range(1):
        polarizabilities[i, j] = np.dot(dipole_tensors[i], responses[j])

In [60]:
print(polarizabilities)

[[-1.996  0.     0.   ]
 [ 0.     0.     0.   ]
 [ 0.     0.     0.   ]]


## References

Trygve Helgaker, Poul Jørgensen, Jeppe Olsen, _Molecular Electronic-Structure Theory_ (John Wiley & Sons, Chichester, 2000). Available in digital form at [doi:10.1002/9781119019572](https://doi.org/10.1002/9781119019572).

Julien Toulouse, _Introduction to the calculation of molecular properties by response theory_, lecture notes for a class given at the Université Pierre et Marie Curie, 2017. Last accessed on March 11, 2017 at [http://www.lct.jussieu.fr/pagesperso/toulouse/enseignement/molecular_properties.pdf](http://www.lct.jussieu.fr/pagesperso/toulouse/enseignement/molecular_properties.pdf).

Trygve Helgaker, _Time-independent molecular properties_, lecture at the 13th Sostrup Summer School Quantum Chemistry and Molecular Properties, 2014. Last accessed on March 11, 2017 at [http://folk.uio.no/helgaker/talks/SostrupTI_14.pdf](http://folk.uio.no/helgaker/talks/SostrupTI_14.pdf).

Poul Jørgensen, _Molecular and Atomic Applications of Time-Dependent Hartree-Fock Theory_, Annual Review of Physical Chemistry __26__, 359-380 (1975). Available in digital form at [doi:10.1146/annurev.pc.26.100175.002043](https://doi.org/10.1146/annurev.pc.26.100175.002043).